In [1]:
import sys
sys.path.append('../')
import functions
%run functions.py
#exec(open('./functions.py', encoding="utf8").read())

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.59 seconds)
Writing emoji data to C:\Users\Annelie Schridde\.demoji\co

In [21]:
df = pickle.load(open('data_n_models/eng_df_labelled.pkl', 'rb'))
df.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,clean,clean_vader,compound,lem,lemma
0,383,0,1323291682000000000,133,GreenerRetail,Who and what has made our greener retailing tr...,0,Planet Earth,05-Mar-2021,70,False,en,@zalando,NaN,NaN,greener retailing tracker week step forward,Who and what has made our greener retailing tr...,0.0000,"(greener, retailing, tracker, week, step, forw...",greener retailing tracker week step forward
1,384,1,1259761930000000000,1,fattybabycat,".@Zalando offers nothing more than scripted, c...",0,,05-Mar-2021,0,False,en,@zalando,NaN,NaN,offers scripted cordial replies instead actual...,"offers nothing more than scripted, cordial rep...",-0.9060,"(offers, scripted, cordial, replies, instead, ...",offer script cordial reply instead actual cust...
2,385,2,835467350200000128,2296,piotrkarwatka,We've got a great talk with Fabian Wesner on h...,0,"Wrocław, Polska",05-Mar-2021,1347,False,en,@zalando,NaN,NaN,got great talk fabian wesner new venture roq t...,We've got great talk with Fabian Wesner his ne...,0.6249,"(got, great, talk, fabian, wesner, new, ventur...",get great talk fabian wesner new venture roq t...
3,386,3,58485935,114,MissyDawn27586,@Zalando recieved wrong item... don’t know how...,0,,05-Mar-2021,27,False,en,@zalando,NaN,NaN,recieved wrong item don’t know item cheaper it...,recieved wrong item... don’t know how about th...,-0.4767,"(recieved, wrong, item, do, n’t, know, item, c...",recieve wrong item do n’t know item cheap item...
4,387,4,1059339104000000000,11725,CarolCooney7,@DrewLawDesign @Zalando have sent me a few de...,0,,05-Mar-2021,457,False,en,@zalando,NaN,NaN,sent deliveries paper bags,have sent few deliveries paper bags now.,0.0000,"(sent, deliveries, paper, bags)",send delivery paper bag


In [35]:
df = df.sample(frac=1, random_state= 42)

In [36]:
df["compound"] = np.where(df["compound"] > 0, 1, df["compound"])      # -1 means positive
df["compound"] = np.where(df["compound"] == 0, 0, df["compound"])     # 0 means neutral
df["compound"] = np.where(df["compound"] < 0, -1, df["compound"])      # 1 means negative

In [37]:
# split data into 3 companies
df_zalando = df[df['keyword'].isin(['@zalando', 'zalando']) ]
df_boohoo = df[df['keyword'].isin(['boohoo', 'bohoo']) ]
df_asos = df[df['keyword'].isin(['asos']) ]
print(df_zalando.shape)
print(df_boohoo.shape)
print(df_asos.shape)

(10334, 20)
(36653, 20)
(63000, 20)


In [38]:
df_zal_sort = df_zalando.sort_values('compound')
df_bo_sort = df_boohoo.sort_values('compound')
df_asos_sort = df_asos.sort_values('compound')

In [66]:
df_zal_sort.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,clean,clean_vader,compound,lem,lemma
13537,21929,158,1125728514000000000,9661,DommeLineCoUk,@missbellalugosi @Zalando Problem starts at Ad...,0,,06-Mar-2021,2196,False,en,zalando,1.0,1.368196e+18,problem starts adidas selling latex websites,Problem starts Adidas. They're selling latex t...,-1.0,"(problem, starts, adidas, selling, latex, webs...",problem start adida sell latex website
3388,3771,284,1306299552000000000,1051,daniel4206969,@EtherealScripts really want to join. i recent...,0,,26-Feb-2021,21,False,en,zalando,NaN,NaN,want join recently joined goatify combination ...,"really want join. recently joined goatify, and...",-1.0,"(want, join, recently, joined, goatify, combin...",want join recently join goatify combination et...
20823,34586,318,1116295451000000000,313,NoCountry4Kids,What a morning! Went all in for UNC on Zalando...,6,,09-Mar-2021,88,False,en,zalando,49.0,NaN,morning went unc zalando hit started pinging d...,What morning! Went all for UNC Zalando but hit...,-1.0,"(morning, went, unc, zalando, hit, started, pi...",morning go unc zalando hit start ping dunk loa...
70976,112008,193,1015684290,32054,CreepItReal_,My zalando package didn't arrive because the a...,0,Niederlande,25-Mar-2021,650,False,en,zalando,0.0,NaN,zalando package arrive adress wasn correct fun...,zalando package didn't arrive because the adre...,-1.0,"(zalando, package, arrive, adress, wasn, corre...",zalando package arrive adress wasn correct fun...
44867,73448,212,1321312433000000000,1830,thebeachworld,@ZalandoA haselts solution @haselts \ni missed...,0,Germany,20-Mar-2021,12,False,en,zalando,1.0,1.373327e+18,haselts solution missed drop jordan unc face t...,haselts solution missed the last drop jordan u...,-1.0,"(haselts, solution, missed, drop, jordan, unc,...",haselt solution miss drop jordan unc face tear...


In [73]:
#looking at negative tweets

df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[0] 

"@missbellalugosi @Zalando Problem starts at Adidas. They're selling it as latex on their websites."

In [92]:
print("1: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[1])
print("2: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[2])
print("3: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[3]) #rather positive than negative
print("4: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[4])
print("5: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[5])
print("6: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[6])
print("7: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[7])
print("8: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[8])
print("9: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[9]) 
print("10: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[10])
print("11: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[11])
print("12: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[12]) # advertising
print("13: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[13]) # high return rates - environment concerns?
print("14: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[14])
print("15: ",df_zal_sort[df_zal_sort.compound == -1].tweet.reset_index().tweet[15])

# problems with deliveries and articles out of stock - probably bots?

1:  @EtherealScripts really want to join. i recently joined goatify, and in combination with ethereal that would be a killer combo. i see all other members cooking, but im standing there with not a single good script for zalando, snipes and solebox... so yeah i really want ethereal!
2:  What a morning! Went all in for UNC on Zalando but hit 0
@thesoleradar and @BestNotify_ started pinging that the dunks were loaded on so I quickly swapped out csv and @theGaneshBot did it's magic! I'm still shaking. 63 pairs in total

@mountainproxies @BelgianChefs @jakescotting https://t.co/fchVvn7zrc
3:  My zalando package didn't arrive because the adress wasn't correct. Which is funny because this was the second package of the same order and the first package arrived just two days earlier...
4:  @ZalandoA haselts solution @haselts 
i missed the last drop of jordan unc 😂
because rn i have no cart able account for zalando
do you can help me?
5:  @its_seesaw zalando plus is both a curse and blessing,, m

In [93]:
print("1: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[1])
print("2: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[2])
print("3: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[3]) 
print("4: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[4]) # postive but should be negative!!
print("5: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[5])
print("6: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[6])
print("7: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[7]) # rather negative
print("8: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[8])
print("9: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[9]) 
print("10: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[10]) # limited clothing
print("11: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[11])
print("12: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[12]) 
print("13: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[13]) 
print("14: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[14]) # destroying Zalando?
print("15: ",df_zal_sort[df_zal_sort.compound == 1].tweet.reset_index().tweet[15])

#starlabscripts seems to be abig site for gathering huge amounts of articles within minutes.

1:  @STARLABScripts DAMNNN LETS TRY ONE MORE TIME. I NEED YOU FOR ZALANDO MY LOVELY FRIENDS!
2:  @Charon_AIO Dunks on Zalando !✌🏻 https://t.co/CAeRcZk6Wm
3:  Discounted from Zalando @snatchtool https://t.co/hQghdzfj0u
4:  @Zalando 
...a spoke in the wheel, when they make a mistake.

What was supposed to be a great shopping experience turned into a battle to get my money back.

#Zalando 
#BadCustomerService
#Awfulservice
#Fail 
#NoCustomerService 
#Inefficient
#NoAnswer
#Disappointing
#BadService
5:  Zalando hit a 25% growth spurt in 2020 and outgrew its ERP system! To better serve its 35 million customers, the company digitally transformed its innovative fashion platform with SAP #S4HANA to support ambitious growth. https://t.co/t6gagOiH3b https://t.co/L1FzAAjm5F
6:  @Zalando Thank you
7:  @JohnMatchett6 @3_stripes4life @Zalando I’m the same love  a pair but these soles look like there turning into Stilton cheese 😬
8:  Very throughout guideline  &gt;Zalando RESTful API and Event Scheme

In [95]:
print("1: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[1]) # not relevant
print("2: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[2]) # not nice clothing
print("3: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[3]) 
print("4: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[4]) 
print("5: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[5])
print("6: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[6])
print("7: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[7]) # collab
print("8: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[8])
print("9: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[9]) # human rights
print("10: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[10])
print("11: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[11])
print("12: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[12]) 
print("13: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[13]) # not good collection
print("14: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[14]) 
print("15: ",df_bo_sort[df_bo_sort.compound == -1].tweet.reset_index().tweet[15]) # failures in supply chain

1:  @Shannon66918133 @DonaldJTrumpJr Boohoo I want the Democrats control of everything forever boohoo I wanted them wipe my ass for me cuz I'm a little tiny baby
2:  Boohoo got some ugly ass clothes
3:  @ThegirlJT @boohoo My delusional ass bought everything in a size 12 knowing damn well I'm big and pregnant 😭😭
4:  @JeffreyMyles3 @b1rd_tweets @rnassau7 @by_justin Boohoo. Being a veteran doesn't make anybody special, nor is it something to base one's identity around.
5:  @onewhoseethall @LGBTSRH @emmakennytv "Waaahh boohoo trans people bad" isn't criticism.
6:  Paint to go with a poem, all cracked this morn, and lets not mention the handprints etc😢 boohoo to painting -tough gig 🤨🤔 https://t.co/hPDQ5jgx1b
7:  I’m so sad about the City Girls Boohoo collab
8:  @Numbers28 Oh boohoo. If he was the least bit concerned about his family he would have kept his ass home.
9:  @FNW_US And how will BooHoo support the previously exploited garment workers? Underpaid from the start and now presumably w

In [97]:
print("1: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[1]) # recommendation
print("2: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[2])
print("3: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[3]) 
print("4: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[4]) 
print("5: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[5]) # recommendation
print("6: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[6])
print("7: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[7]) 
print("8: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[8]) # prob nice article 
print("9: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[9]) 
print("10: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[10]) 
print("11: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[11])
print("12: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[12]) 
print("13: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[13]) # not positive
print("14: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[14]) 
print("15: ",df_bo_sort[df_bo_sort.compound == 1].tweet.reset_index().tweet[15])

1:  Pretty little thing , fashion nova or boohoo 🤦🏾‍♀️
2:  @boohoo OMGGGG! i was placing my order &amp; going “WTF IS THIS!” 🤣🤣🤣🤣 i got FOOLED! well done boohoo!🤣🤣👏🏼 @boohoo_cshelp @boohoo #igotfooledbyboohoo #AprilFoolsDay #aprilfools
3:  @boohoo Cosmo 🐶 rescued 3 years ago #NationalPetDay https://t.co/8CQdBxtqwg
4:  @narayansmate Well the Chinese will miss out on the best wine in the world! Boohoo! Another countries gain!!
5:  @draeee__ try pretty little thing, boohoo, h&amp;m, missguided, shein, zara but for them go into the store.
6:  @mtgreenee And the only thing you are pushing is GUNS! Guns, guns, guns, they want to take my guns, guns, guns BooHoo!

#ExpelMarjorieTaylorGreene
7:  Boohoo better count their fucking days
8:  If anyone finds these please send 10
Boxes to my house or at least tell me where I can get them 😍 https://t.co/mDSizComu1
9:  @froydnf Skeppy but before dream smp
10:  @HyperbolicLens @Rachael_Swindon @OG_Goldinhart they don't? i haven't traveled overseas in 40

In [107]:
print("1: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[1]) 
print("2: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[2]) 
print("3: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[3]) # bad customer service
print("4: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[4]) # need a restock
print("5: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[5]) # actually positive
print("6: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[6])
print("7: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[7]) 
print("8: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[8])
print("9: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[9]) 
print("10: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[10])
print("11: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[11]) # delivery
print("12: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[12]) 
print("13: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[13])
print("14: ",df_asos_sort[df_asos_sort.compound == -1].tweet.reset_index().tweet[14]) # technical problems


1:  *regrets not booking anywhere for dinner*
2:  JORDAN ASOS MT Mar 29 Climate: High: 58 Low: 21 Precip: 0.02 Snow: Missing https://t.co/OxJQ6NjwdE https://t.co/iWvvCyUh37
3:  @ASOS been trying to speak to you on the live chat for 3 days now what’s going on ??? I need to contact you about a missing parcel
4:  @ASOS @ASOS_HeretoHelp pls restock these im so obsessed 😪😪 https://t.co/YU4Yma8VS3
5:  @spicyred11 bet, ASOS got some good shit imo
6:  @SCienceSophs @ASOS @ASOS_HeretoHelp Ah ffs! And to think that this time last year, they were all about looking after healthcare workers https://t.co/O8QtAjqHdw
7:  https://t.co/lXSpdazbeA In case you missed it yesterday, new track, new alias...ENJOY! #ASoS
8:  #MyASOSLuck I’m unlucky because I never win any of these competitions 👎🏽👎🏽👎🏽 https://t.co/hvYtt7zJcz
9:  @jongaunt @gresham_garry I thought she wasn’t allowed back after joining ASOS?

I think I may have got this wrong...
10:  These boots from asos are brand new  and boy got me on this dir

In [108]:
print("1: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[1]) 
print("2: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[2]) 
print("3: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[3]) 
print("4: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[4]) 
print("5: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[5]) 
print("6: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[6])
print("7: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[7]) # negative instead of positive
print("8: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[8])
print("9: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[9]) # great dresses
print("10: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[10])
print("11: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[11]) 
print("12: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[12]) # from asos itself, i guess
print("13: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[13]) 
print("14: ",df_asos_sort[df_asos_sort.compound == 1].tweet.reset_index().tweet[14]) # from asos itself, i guess


1:  @faps2finn Oh wow I recognize his face from Asos
2:  @ApparentlyWeRun @ASOS Oh no 🤦‍♀️😔
3:  id just buy at ASOS if theyre #vaccinated like me https://t.co/GghMQxVuyj😷😷😷😗
4:  @ASOS https://t.co/xicOiNbWmC All this time being confident with my own body, support young confident women 💜✊🏼 #ASOSIWD21
5:  GIFTCARDS GIFTCARDS

I want some gift cards
Amazon, ASOS, SHEIN, and any others you can think of I want them all

|| findom finsub sub dom cuckold sissy feet stomping simp luxury paypig humanatm sph whalesub || https://t.co/zzYBD7enz3
6:  @ZombieTrev5k @NWSVegas @ToddLericos @DSBerc @WarnJenn @ChelseaKryston @TeddFlorendo @sherryswensk @SallyjTV @8NewsNow Um. Have you met my friend #ASOS (who may or may not be related to @iembot_vef) ?
7:  @ASOS why are you promoting obesity?
8:  So good I had to share! Check out all the items I'm loving on @Poshmarkapp from @LindsNitz #poshmark #fashion #style #shopmycloset #pinkvictoriassecret #seanjohn #asos: https://t.co/DljTtSRWyR https://t.co/z15l